# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
#conda install -c pyviz hvplot geoviews

In [2]:
#pip install cartopy

In [21]:
#!pip install GeoViews

In [53]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import cartopy.crs as ccrs

# Import API key
from api_keys import geoapify_key

In [54]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,happy valley-goose bay,53.3168,-60.3315,276.32,93,100,5.14,CA,1684976961
1,1,port-aux-francais,-49.3500,70.2167,279.25,99,100,6.52,TF,1684976962
2,2,saint-pierre,-21.3393,55.4781,295.97,64,100,1.54,RE,1684976962
3,3,olonkinbyen,70.9221,-8.7187,271.16,89,100,18.02,SJ,1684976962
4,4,ugoofaaru,5.6667,73.0000,301.99,72,12,3.11,MV,1684976963


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles ='OSM',
    size = 'Humidity'
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [61]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[
    (city_data_df["Max Temp"] >275) 
    & (city_data_df["Max Temp"] <= 290)
    & (city_data_df["Wind Speed"] < 5)
    & (city_data_df["Cloudiness"] < 50)
]

# Drop any rows with null values
ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,nyagan,62.1406,65.3936,286.70,77,0,1.92,RU,1684976965
13,13,kushiro,42.9750,144.3747,287.20,63,0,3.01,JP,1684976925
16,16,kasungu,-13.0333,33.4833,287.83,75,0,1.55,MW,1684976966
23,23,anadyr,64.7500,177.4833,279.43,70,20,3.00,RU,1684976968
27,27,salekhard,66.5300,66.6019,285.44,59,7,3.87,RU,1684976969
39,39,waitangi,-43.9535,-176.5597,285.27,71,29,1.79,NZ,1684976972
47,47,bilibino,68.0546,166.4372,282.88,46,17,1.63,RU,1684976974
57,57,colonia,50.9333,6.9500,283.13,86,0,1.54,DE,1684976969
60,60,diego de almagro,-26.3667,-70.0500,287.94,52,0,0.82,CL,1684976978
69,69,akureyri,65.6835,-18.0878,275.82,76,44,4.38,IS,1684976980


### Step 3: Create a new DataFrame called `hotel_df`.

In [62]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df.head(10)

/var/folders/sr/q8_2cwcj1sd2hyxcgdvpgjmr0000gn/T/ipykernel_86421/2612808406.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
11,nyagan,RU,62.1406,65.3936,77,
13,kushiro,JP,42.9750,144.3747,63,
16,kasungu,MW,-13.0333,33.4833,75,
23,anadyr,RU,64.7500,177.4833,70,
27,salekhard,RU,66.5300,66.6019,59,
39,waitangi,NZ,-43.9535,-176.5597,71,
47,bilibino,RU,68.0546,166.4372,46,
57,colonia,DE,50.9333,6.9500,86,
60,diego de almagro,CL,-26.3667,-70.0500,52,
69,akureyri,IS,65.6835,-18.0878,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
category= 'accommodation.hotel'
params = {
    "categories":category,
    "limit": limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)   

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nyagan - nearest hotel: Старый Замок
kushiro - nearest hotel: ANA Hotel Kushiro
kasungu - nearest hotel: Kasungu Inn
anadyr - nearest hotel: Гостевой дом
salekhard - nearest hotel: Гостиница
waitangi - nearest hotel: Hotel Chathams
bilibino - nearest hotel: No hotel found
colonia - nearest hotel: Wasserturm Hotel Cologne
diego de almagro - nearest hotel: Residencial Hambyll
akureyri - nearest hotel: Hótel Norðurland
yellowknife - nearest hotel: No hotel found
yermentau - nearest hotel: Достык
houghton - nearest hotel: Quality Inn & Suites
constantia - nearest hotel: Hotel Chérica
kurtamysh - nearest hotel: Загородный Дом Ласточкино Гнездо
khovd - nearest hotel: Khovd hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
stanisic - nearest hotel: No hotel found
st. john's - nearest hotel: Alt Hotel
yuzhno-kurilsk - nearest hotel: Dom Druzhby
labrador city - nearest hotel: Carol Inn
myrina - nearest hotel: Διαμαντίδης
kurumkan - nearest hotel: Туяа
dunedin - n

,City,Country,Lat,Lng,Humidity,Hotel Name
11,nyagan,RU,62.1406,65.3936,77,Старый Замок
13,kushiro,JP,42.9750,144.3747,63,ANA Hotel Kushiro
16,kasungu,MW,-13.0333,33.4833,75,Kasungu Inn
23,anadyr,RU,64.7500,177.4833,70,Гостевой дом
27,salekhard,RU,66.5300,66.6019,59,Гостиница
...,...,...,...,...,...,...
547,grand falls-windsor,CA,48.9332,-55.6649,60,Hotel Robin Hood
548,zhanga buqtyrma,KZ,49.6295,83.5248,68,Gelios
554,kandava,LV,57.0409,22.7747,82,Kandava
557,juva,RU,56.4368,57.9478,57,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [65]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles = 'OSM',
    c = "Country",
    hover_cols = 'all',
)
# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,index,City,Humidity,Hotel Name)